## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from itertools import product

from hkexoi import *
from hkexoi.hktradedates import *
from hkexoi.vixcalc import *
from hkexoi.database import *
from bsoption.bsmodel import *

## Components

### Monthly option expiry dates

In [2]:
startdt = datetime(2018, 5, 1)
enddt = datetime(2022, 4, 30)

expdatelist = [date for date in setdatelist if (date >= startdt) and (date <= enddt)]
expdatelist[-12:]

[datetime.datetime(2021, 5, 28, 0, 0),
 datetime.datetime(2021, 6, 29, 0, 0),
 datetime.datetime(2021, 7, 29, 0, 0),
 datetime.datetime(2021, 8, 30, 0, 0),
 datetime.datetime(2021, 9, 29, 0, 0),
 datetime.datetime(2021, 10, 28, 0, 0),
 datetime.datetime(2021, 11, 29, 0, 0),
 datetime.datetime(2021, 12, 30, 0, 0),
 datetime.datetime(2022, 1, 28, 0, 0),
 datetime.datetime(2022, 2, 25, 0, 0),
 datetime.datetime(2022, 3, 30, 0, 0),
 datetime.datetime(2022, 4, 28, 0, 0)]

### Futures OHLC query

In [5]:
num = 12
expdt0 = expdatelist[-num]
expdt1 = expdatelist[-num+1]
expstr0 = expdt0.strftime('%Y-%m-%d')
expstr1 = expdt1.strftime('%Y-%m-%d')
yrmth0 = (expdt0.year, expdt0.month)
yrmth1 = (expdt1.year, expdt1.month)

ftinputdict0 = {'style': 'regular',  'month': [yrmth1],  'asset': 'HSI', 'startdate': expstr0,  'enddate': expstr1}

dfft0 = loadfuturedata(ftinputdict0)
dfft0

,tradedate,month,td_open,td_high,td_low,td_close,td_vol,oi
0,2021-05-28,2021-06-29,29100,29253,28950,28986,94726,107163
1,2021-05-31,2021-06-29,29184,29184,28791,28958,86187,106676
2,2021-06-01,2021-06-29,28930,29400,28923,29350,104308,109273
3,2021-06-02,2021-06-29,29312,29398,29081,29144,85440,107049
4,2021-06-03,2021-06-29,29201,29280,28801,28858,99136,103684
5,2021-06-04,2021-06-29,28792,28933,28632,28806,89186,101663
6,2021-06-07,2021-06-29,28880,28938,28528,28748,82770,101522
7,2021-06-08,2021-06-29,28755,28924,28559,28683,81319,101754
8,2021-06-09,2021-06-29,28743,28809,28617,28710,74005,102044
9,2021-06-10,2021-06-29,28661,28895,28581,28608,77627,103415


### Option chain query

In [8]:
tdstr1 = dfft0.loc[2, 'tradedate'].strftime('%Y-%m-%d')

opinputdict1 = {'style': 'regular',  # trading hour style: regular, night, settle or entire
                'month': [yrmth1], # list of option expiry month in the form (yyyy, mm)
                'asset': 'HSI',      # asset symbol ("HSI" or "HHI", must be upper)
                'strike_lowerbound': 12000,  # Lower bound for range of strike prices (inclusive)
                'strike_upperbound': 36000,  # Upper bound for range of strike prices (inclusive)
                'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
                'startdate': tdstr1,  # query period startdate (inclusive)
                'enddate': tdstr1  # query period startdate (inclusive) 
    }

orderfield = ('tradedate', 'month', 'optype', 'strike', )

dfop1 = loadopdata(opinputdict1, orderfield)
lobound = 10
upbound = 1000
dfopbound1 = dfop1[(dfop1['td_close'] >= lobound) & (dfop1['td_close'] <= upbound)]
dfopbound1

,tradedate,month,strike,optype,td_open,td_high,td_low,td_close,td_vol,oi,iv,oi
55,2021-06-01,2021-06-29,26200,P,25,29,22,12,48,2166,20,2166
56,2021-06-01,2021-06-29,26400,P,29,32,26,16,64,1370,19,1370
57,2021-06-01,2021-06-29,26600,P,35,41,31,21,74,1478,19,1478
58,2021-06-01,2021-06-29,26800,P,42,50,35,27,127,1638,19,1638
59,2021-06-01,2021-06-29,27000,P,50,61,42,36,356,2898,19,2898
60,2021-06-01,2021-06-29,27200,P,62,73,50,46,120,1823,18,1823
61,2021-06-01,2021-06-29,27400,P,79,91,60,60,471,1948,18,1948
62,2021-06-01,2021-06-29,27600,P,98,113,72,77,479,1306,18,1306
63,2021-06-01,2021-06-29,27800,P,118,141,88,95,385,1467,17,1467
64,2021-06-01,2021-06-29,28000,P,150,177,109,118,1017,1920,17,1920


In [ ]:
dfputnext0 = dfotmnext0[dfotmnext0['optype'] == 'P']
dfputnext0